In [1]:
import open3d as o3d
import numpy as np
import os

Convert D-FAUST dataset into .ply file

In [2]:
file_path = "/usr/stud/srinivaa/code/CaDeX/resource/data/Humans/D-FAUST/50002_hips/pcl_seq/00000003.npz"
data = np.load(file_path)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data['points'])
o3d.io.write_point_cloud("data_3.ply", pcd)

True

Get the attributes of pcl_seq

In [3]:
for i in data.files:
    print(i)

points
loc
scale


Get the attributes of points_seq

In [4]:
points_seq = np.load("/usr/stud/srinivaa/code/CaDeX/resource/data/Humans/D-FAUST/50002_chicken_wings/points_seq/00000000.npz")
for i in points_seq.files:
    print(i)

points
occupancies
loc
scale


Check the size of total dataset

In [34]:
path, dirs, files = next(os.walk("/usr/stud/srinivaa/code/CaDeX/resource/data/Humans/D-FAUST"))
total_file_size = 0

for i in range(len(dirs)): 
    if(dirs[i] != '50004_chicken_wings' and dirs[i] != '50020_hips' and dirs[i] != '50025_shake_arms' and dirs[i] != '50020_running_on_spot' and dirs[i] != '50007_light_hopping_loose'):
        path_i,dirs_i,files_i = next(os.walk(os.path.join(path,dirs[i],'points_seq')))
        total_file_size += len(files_i)

In [35]:
total_file_size

39920

In [32]:
path, dirs, files = next(os.walk("/usr/stud/srinivaa/code/CaDeX/resource/data/Humans/D-FAUST"))
total_file_size = 0

for i in range(len(dirs)): 
    if(dirs[i] != '50002_light_hopping_loose' and dirs[i] != '50004_punching' and dirs[i] != '50007_shake_shoulders' and dirs[i] != '50009_chicken_wings' and dirs[i] != '50020_chicken_wings' and dirs[i] != '50022_light_hopping_loose' and dirs[i] != '50025_light_hopping_loose' and dirs[i] != '50026_shake_arms' and dirs[i] != '50027_shake_shoulders' and dirs[i] != '50004_chicken_wings' and dirs[i] != '50020_hips' and dirs[i] != '50025_shake_arms' and dirs[i] != '50020_running_on_spot' and dirs[i] != '50007_light_hopping_loose'):
        path_i,dirs_i,files_i = next(os.walk(os.path.join(path,dirs[i],'points_seq')))
        total_file_size += len(files_i)

In [33]:
total_file_size

37867

## Custom dataset of CaDeX

In [2]:
import dataset.oflow_dataset as oflow_dataset

In [3]:
def get_transforms():
    """Returns transform objects.

    Args:
        cfg (yaml config): yaml config object
    """
    n_pcl = 100
    n_pt = 512
    n_pt_eval = 10000

    transf_pt = oflow_dataset.SubsamplePoints(n_pt)
    transf_pt_val = oflow_dataset.SubsamplePointsSeq(n_pt_eval, random=False)
    transf_pcl_val = oflow_dataset.SubsamplePointcloudSeq(n_pt_eval, random=False)
    transf_pcl = oflow_dataset.SubsamplePointcloudSeq(n_pcl, connected_samples=True)

    return transf_pt, transf_pt_val, transf_pcl, transf_pcl_val

In [4]:
def get_data_fields(mode):
    """Returns data fields.

    Args:
        mode (str): mode (train|val|test)
        cfg (yaml config): yaml config object
    """
    fields = {}
    seq_len_train = 17
   
    seq_len_val = seq_len_train
    p_folder = "points_seq" # points_seq: contains information regarding the points and their corresponding occupancy values
    pcl_folder = "pcl_seq" #pcl_seq : contains information regarding the points, scale and loc
    mesh_folder = "mesh_seq" #mesh_seq: non-existent. Utilize this to have a file containing points and faces for each model.
    generate_interpolate = False #False
    unpackbits = True # True
    
    training_all = False
    
    n_training_frames = 8

    # Transformation
    transf_pt, transf_pt_val, transf_pcl, transf_pcl_val = get_transforms()

    # Fields
    pts_iou_field = oflow_dataset.PointsSubseqField
    pts_corr_field = oflow_dataset.PointCloudSubseqField

    # MeshSubseqField can be used to load mesh fields

  
    not_choose_last = False
    training_multi_files = False
    
    loss_recon = "true"
    loss_corr = "true"

    if mode == "train":
        if loss_recon:
            if training_all:
                fields["points"] = pts_iou_field(
                    p_folder,
                    transform=transf_pt,
                    all_steps=True,
                    seq_len=seq_len_train,
                    unpackbits=unpackbits,
                    use_multi_files=training_multi_files,
                )
            else:
                fields["points"] = pts_iou_field(
                    p_folder,
                    sample_nframes=n_training_frames,
                    transform=transf_pt,
                    seq_len=seq_len_train,
                    fixed_time_step=0,
                    unpackbits=unpackbits,
                    use_multi_files=training_multi_files,
                )
            fields["points_t"] = pts_iou_field(
                p_folder,
                transform=transf_pt,
                seq_len=seq_len_train,
                unpackbits=unpackbits,
                not_choose_last=not_choose_last,
                use_multi_files=training_multi_files,
            )


            fields["mesh"] = oflow_dataset.MeshField(
            mesh_folder, seq_len=seq_len_val)

    # only training can be boost by multi-files
    # modify here, if not train, val should also load the same as the test
    else:
        fields["points"] = pts_iou_field(
            p_folder,
            transform=transf_pt_val,
            all_steps=True,
            seq_len=seq_len_val,
            unpackbits=unpackbits,
        )
        fields[
            "points_mesh"
        ] = pts_corr_field(  # ? this if for correspondence? Checked, this is for chamfer distance, make sure that because here we use tranforms, teh pts in config file must be 100000
            pcl_folder, transform=transf_pcl_val, seq_len=seq_len_val
        )
    # Connectivity Loss:
    if loss_corr:
        # fields["pointcloud"] = pts_corr_field(
        #     pcl_folder,
        #     transform=transf_pcl,
        #     seq_len=seq_len_train,
        #     use_multi_files=training_multi_files,
        # )
        fields["pointcloud"] = oflow_dataset.MeshField(
            mesh_folder, seq_len=seq_len_val)
    if mode == "test" and generate_interpolate:
        fields["mesh"] = oflow_dataset.MeshSubseqField(
            mesh_folder, seq_len=seq_len_val, only_end_points=True
        )
    fields["oflow_idx"] = oflow_dataset.IndexField()
    return fields

In [5]:
fields = get_data_fields("train")

In [6]:
dataset_folder = "/usr/data/cvpr_shared/marvin/Data/CaDeX/data/Humans"
categories = ["D-FAUST"]

In [7]:
dataset = oflow_dataset.HumansDataset(
        dataset_folder,
        fields,
        split="train",
        categories=categories,
        length_sequence=17,
        n_files_per_sequence=-1,
        offset_sequence=15,
        ex_folder_name="mesh_seq",
    )

In [8]:
dataset.fields['mesh'].load("/usr/data/cvpr_shared/marvin/Data/CaDeX/data/Humans/D-FAUST/50002_chicken_wings",0,0,15)

AttributeError: module 'numpy' has no attribute 'arrray'

In [9]:
dataset.__getitem__(1)

WARNING - 2022-09-08 20:44:47,528 - subseq_dataset - Error occured when loading field mesh of model 50002_one_leg_loose
